In [1]:
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
sales_houses = pd.read_csv("../inputs/houses_Madrid.csv")

In [3]:
sales_houses.head()

,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,...,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,...,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,4,21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,...,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


In [4]:
# Vamos a comprobar las columnas por las que está compuesto nuestro DataFrame.

sales_houses.columns

Index(['Unnamed: 0', 'id', 'title', 'subtitle', 'sq_mt_built', 'sq_mt_useful',
       'n_rooms', 'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'latitude',
       'longitude', 'raw_address', 'is_exact_address_hidden', 'street_name',
       'street_number', 'portal', 'floor', 'is_floor_under', 'door',
       'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area',
       'is_rent_price_known', 'buy_price', 'buy_price_by_area',
       'is_buy_price_known', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'built_year', 'has_central_heating',
       'has_individual_heating', 'are_pets_allowed', 'has_ac',
       'has_fitted_wardrobes', 'has_lift', 'is_exterior', 'has_garden',
       'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room',
       'is_furnished', 'is_kitchen_equipped', 'is_accessible',
       'has_green_zones', 'energy_certificate', 'has_parking',
       'has_private_parking', 'has_public_parking',
       'is_parking_included_in_price', 'parki

In [5]:
# Seleccionamos aquellas columnas que más nos interesan para nuestro predicción de precios.

sales_houses = sales_houses[["neighborhood_id", "house_type_id", "is_renewal_needed", "is_new_development",
                             "sq_mt_built","n_rooms", "n_bathrooms", "floor", "is_exterior", "has_lift",
                             "has_parking", "has_storage_room", "has_terrace", "has_balcony", 
                             "has_pool", "buy_price", "buy_price_by_area"]]
sales_houses.head()

,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area
0,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,HouseType 1: Pisos,False,False,64.0,2,1.0,3,True,False,False,NaN,NaN,NaN,NaN,85000,1328
1,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,HouseType 1: Pisos,True,False,70.0,3,1.0,4,True,True,False,NaN,True,NaN,NaN,129900,1856
2,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,94.0,2,2.0,1,True,True,False,True,NaN,NaN,NaN,144247,1535
3,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,True,True,False,True,NaN,NaN,NaN,109900,1717
4,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,HouseType 1: Pisos,False,False,108.0,2,2.0,4,True,True,True,True,NaN,NaN,True,260000,2407


In [6]:
# Nuestro tasador se va a basar en viviendas plurifamiliares por lo que eliminaremos todas aquellas tipologías
# que no se correspondan con este tipo de viviendas.
# Vamos a comprobar por tanto que tipologías de viviendas tenemos en nuestro DataFrame.

sales_houses.house_type_id.value_counts()

HouseType 1: Pisos            17705
HouseType 2: Casa o chalet     1938
HouseType 5: Áticos            1032
HouseType 4: Dúplex             676
Name: house_type_id, dtype: int64

In [7]:
# Eliminamos las viviendas de tipología tipo 2:

sales_houses = sales_houses.drop(sales_houses[sales_houses["house_type_id"]=="HouseType 2: Casa o chalet"].index).reset_index()

In [8]:
# Vamos a comprobar ahora que tipo de datos tenemos 

sales_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19804 entries, 0 to 19803
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               19804 non-null  int64  
 1   neighborhood_id     19804 non-null  object 
 2   house_type_id       19413 non-null  object 
 3   is_renewal_needed   19804 non-null  bool   
 4   is_new_development  18883 non-null  object 
 5   sq_mt_built         19803 non-null  float64
 6   n_rooms             19804 non-null  int64  
 7   n_bathrooms         19790 non-null  float64
 8   floor               19135 non-null  object 
 9   is_exterior         18699 non-null  object 
 10  has_lift            19356 non-null  object 
 11  has_parking         19804 non-null  bool   
 12  has_storage_room    6580 non-null   object 
 13  has_terrace         8222 non-null   object 
 14  has_balcony         2942 non-null   object 
 15  has_pool            3921 non-null   object 
 16  buy_

In [9]:
# Vamos a empezar por rellenar los valores NaN de aquellas columnas que tienen menos datos.
# Para ello, vamos a empezar comprobando los valores de dichas columnas

columnas = ["has_storage_room", "has_terrace", "has_balcony", "has_pool"]
for col in columnas:
    print(sales_houses[col].value_counts())

True    6580
Name: has_storage_room, dtype: int64
True    8222
Name: has_terrace, dtype: int64
True    2942
Name: has_balcony, dtype: int64
True    3921
Name: has_pool, dtype: int64


In [10]:
# Como podemos comprobar, los valores NaN en todos los casos anteriores pueden ser sustituidos por False.
# Vamos a proceder por tanto a cambiar esos datos por valores False

In [11]:
def fillNaN(columna):
    sales_houses[columna] = sales_houses[columna].fillna(value="False")
    return sales_houses[columna]

fillNaN ("has_storage_room")
fillNaN ("has_terrace")
fillNaN ("has_balcony")
fillNaN ("has_pool")

0        False
1        False
2        False
3        False
4         True
         ...  
19799     True
19800    False
19801     True
19802     True
19803     True
Name: has_pool, Length: 19804, dtype: object

In [12]:
# Comprobamos la columna m2 para ver cual es el valor nulo. 

sq_mt_built_isnull = sales_houses.sq_mt_built.isnull()
sales_houses.loc[sq_mt_built_isnull]

,index,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area
12794,14415,Neighborhood 31: Bernabéu-Hispanoamérica (5170...,HouseType 5: Áticos,True,False,NaN,13,10.0,1,True,True,True,False,True,False,True,8000000,6867


In [13]:
# Vamos a recuperar este dato.
# Para ello, rellenamos el valor nulo con la división de buy_price por buy_price_by_area.

sales_houses["sq_mt_built"] = sales_houses["sq_mt_built"].fillna(value=round(sales_houses["buy_price"]/sales_houses["buy_price_by_area"],1))
sales_houses.loc[sales_houses.index==12794]

,index,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area
12794,14415,Neighborhood 31: Bernabéu-Hispanoamérica (5170...,HouseType 5: Áticos,True,False,1165.0,13,10.0,1,True,True,True,False,True,False,True,8000000,6867


In [14]:
# Vamos a comprobar ahora si podemos realizar lo mismo con las columnas "is_exterior", "has_lift".

nulos = ["is_exterior", "has_lift"]
for nan in nulos:
    print(sales_houses[nan].value_counts())

True     16922
False     1777
Name: is_exterior, dtype: int64
True     14895
False     4461
Name: has_lift, dtype: int64


### En este caso comprobamos que las columnas ya cuentan con valores True y False, por lo que no tenemos forma de averiguar dichos datos.
### Tampoco tenemos forma de rellenar los valores nulos de las columnas "house_type_id", "is_new_development", "n_bathrooms", "floor".
### Eliminamos por tanto, todos los valores nulos de estas columnas.

In [15]:
sales_houses_clean = sales_houses.dropna().reset_index()

In [16]:
# Vamos a comprobar el nº definitivo de datos que tenemos en nuestro dataset.

sales_houses_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17398 entries, 0 to 17397
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   level_0             17398 non-null  int64  
 1   index               17398 non-null  int64  
 2   neighborhood_id     17398 non-null  object 
 3   house_type_id       17398 non-null  object 
 4   is_renewal_needed   17398 non-null  bool   
 5   is_new_development  17398 non-null  object 
 6   sq_mt_built         17398 non-null  float64
 7   n_rooms             17398 non-null  int64  
 8   n_bathrooms         17398 non-null  float64
 9   floor               17398 non-null  object 
 10  is_exterior         17398 non-null  object 
 11  has_lift            17398 non-null  object 
 12  has_parking         17398 non-null  bool   
 13  has_storage_room    17398 non-null  object 
 14  has_terrace         17398 non-null  object 
 15  has_balcony         17398 non-null  object 
 16  has_

### Vamos a obtener ahora la geolocalización de los diferentes vecindarios.

In [17]:
sales_houses_clean.neighborhood_id = sales_houses_clean.neighborhood_id.str.findall(r"(\: \w.* \()(?:.+)(\: \w.*)").map(lambda x: x[-1])

In [18]:
lista = []
def neigh():
    for row in range(0,17398):
        lista.append(sales_houses_clean.neighborhood_id[row][0].lstrip(": ").rstrip(" (") + ", " + sales_houses_clean.neighborhood_id[row][1].lstrip(": ")+"(Madrid)")
    return lista

In [19]:
sales_houses_clean["neighborhood_id"] = sales_houses_clean.apply(lambda row: neigh())

In [20]:
sales_houses_clean.head()

,level_0,index,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area
0,0,0,"San Cristóbal, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,3,True,False,False,False,False,False,False,85000,1328
1,1,1,"Los Ángeles, Villaverde(Madrid)",HouseType 1: Pisos,True,False,70.0,3,1.0,4,True,True,False,False,True,False,False,129900,1856
2,2,2,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,94.0,2,2.0,1,True,True,False,True,False,False,False,144247,1535
3,3,3,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,True,True,False,True,False,False,False,109900,1717
4,4,4,"Los Rosales, Villaverde(Madrid)",HouseType 1: Pisos,False,False,108.0,2,2.0,4,True,True,True,True,False,False,True,260000,2407


In [21]:
coord = sales_houses_clean.neighborhood_id.unique()
coord

array(['San Cristóbal, Villaverde(Madrid)',
       'Los Ángeles, Villaverde(Madrid)',
       'San Andrés, Villaverde(Madrid)',
       'Los Rosales, Villaverde(Madrid)', 'Butarque, Villaverde(Madrid)',
       'Valdebernardo - Valderribas, Vicálvaro(Madrid)',
       'Ambroz, Vicálvaro(Madrid)',
       'Casco Histórico de Vicálvaro, Vicálvaro(Madrid)',
       'El Cañaveral - Los Berrocales, Vicálvaro(Madrid)',
       'Ensanche de Vallecas - La Gavia, Villa de Vallecas(Madrid)',
       'Casco Histórico de Vallecas, Villa de Vallecas(Madrid)',
       'Santa Eugenia, Villa de Vallecas(Madrid)',
       'Orcasitas, Usera(Madrid)', 'San Fermín, Usera(Madrid)',
       'Moscardó, Usera(Madrid)', 'Pradolongo, Usera(Madrid)',
       'Almendrales, Usera(Madrid)', 'Zofío, Usera(Madrid)',
       '12 de Octubre-Orcasur, Usera(Madrid)',
       'Cuzco-Castillejos, Tetuán(Madrid)',
       'Cuatro Caminos, Tetuán(Madrid)', 'Bellas Vistas, Tetuán(Madrid)',
       'Berruguete, Tetuán(Madrid)', 'Valdeacederas

In [22]:
# Inutilizo aquellos barrios en los que geopy no localiza los barrios.


coord1 = ['San Cristóbal, Villaverde(Madrid)',
       'Los Ángeles, Villaverde(Madrid)',
       'San Andrés, Villaverde(Madrid)',
       'Los Rosales, Villaverde(Madrid)', 'Butarque, Villaverde(Madrid)',
       #'Valdebernardo - Valderribas, Vicálvaro(Madrid)',
       'Ambroz, Vicálvaro(Madrid)',
       'Casco Histórico de Vicálvaro, Vicálvaro(Madrid)',
       #'El Cañaveral - Los Berrocales, Vicálvaro(Madrid)',
       #'Ensanche de Vallecas - La Gavia, Villa de Vallecas(Madrid)',
       'Casco Histórico de Vallecas, Villa de Vallecas(Madrid)',
       'Santa Eugenia, Villa de Vallecas(Madrid)',
       'Orcasitas, Usera(Madrid)', 'San Fermín, Usera(Madrid)',
       'Moscardó, Usera(Madrid)', 'Pradolongo, Usera(Madrid)',
       'Almendrales, Usera(Madrid)', 'Zofío, Usera(Madrid)',
       #'12 de Octubre-Orcasur, Usera(Madrid)',
       'Cuzco-Castillejos, Tetuán(Madrid)',
       'Cuatro Caminos, Tetuán(Madrid)', 'Bellas Vistas, Tetuán(Madrid)',
       'Berruguete, Tetuán(Madrid)', 'Valdeacederas, Tetuán(Madrid)',
       'Ventilla-Almenara, Tetuán(Madrid)', 'Niño Jesús, Retiro(Madrid)',
       'Ibiza, Retiro(Madrid)', 'Adelfas, Retiro(Madrid)',
       'Pacífico, Retiro(Madrid)', 'Jerónimos, Retiro(Madrid)',
       'Estrella, Retiro(Madrid)',
       'Palomeras sureste, Puente de Vallecas(Madrid)',
       'Palomeras Bajas, Puente de Vallecas(Madrid)',
       'San Diego, Puente de Vallecas(Madrid)',
       'Numancia, Puente de Vallecas(Madrid)',
       'Entrevías, Puente de Vallecas(Madrid)',
       'Portazgo, Puente de Vallecas(Madrid)', 'Aravaca, Moncloa(Madrid)',
       'Argüelles, Moncloa(Madrid)', 'Casa de Campo, Moncloa(Madrid)',
       'Ciudad Universitaria, Moncloa(Madrid)',
       'Valdemarín, Moncloa(Madrid)', 'Valdezarza, Moncloa(Madrid)',
       'Fontarrón, Moratalaz(Madrid)', 'Vinateros, Moratalaz(Madrid)',
       'Marroquina, Moratalaz(Madrid)', 'Media Legua, Moratalaz(Madrid)',
       'Horcajo, Moratalaz(Madrid)', 'Pavones, Moratalaz(Madrid)',
       'Puerta del Ángel, Latina(Madrid)', 'Aluche, Latina(Madrid)',
       'Lucero, Latina(Madrid)', #'Los Cármenes, Latina(Madrid)',
       'Águilas, Latina(Madrid)', 'Campamento, Latina(Madrid)',
       'Cuatro Vientos, Latina(Madrid)', 'El Plantío, Moncloa(Madrid)',
       'Las Tablas, Fuencarral(Madrid)',
       'Montecarmelo, Fuencarral(Madrid)', 'La Paz, Fuencarral(Madrid)',
       'Pilar, Fuencarral(Madrid)', 'Fuentelarreina, Fuencarral(Madrid)',
       'Peñagrande, Fuencarral(Madrid)',
       'Tres Olivos - Valverde, Fuencarral(Madrid)',
       'Mirasierra, Fuencarral(Madrid)',
       #'Arroyo del Fresno, Fuencarral(Madrid)',
       'El Pardo, Fuencarral(Madrid)', 'Sanchinarro, Hortaleza(Madrid)',
       'Conde Orgaz-Piovera, Hortaleza(Madrid)',
       'Palomas, Hortaleza(Madrid)',
       'Virgen del Cortijo - Manoteras, Hortaleza(Madrid)',
       'Valdebebas - Valdefuentes, Hortaleza(Madrid)',
       'Pinar del Rey, Hortaleza(Madrid)', 'Canillas, Hortaleza(Madrid)',
       'Apóstol Santiago, Hortaleza(Madrid)',
       'Gaztambide, Chamberí(Madrid)',
       'Nuevos Ministerios-Ríos Rosas, Chamberí(Madrid)',
       'Almagro, Chamberí(Madrid)', 'Trafalgar, Chamberí(Madrid)',
       'Arapiles, Chamberí(Madrid)', 'Vallehermoso, Chamberí(Madrid)',
       'Ventas, Ciudad Lineal(Madrid)',
       'Pueblo Nuevo, Ciudad Lineal(Madrid)',
       'Atalaya, Ciudad Lineal(Madrid)',
       'Quintana, Ciudad Lineal(Madrid)',
       'San Juan Bautista, Ciudad Lineal(Madrid)',
       'Colina, Ciudad Lineal(Madrid)',
       'Costillares, Ciudad Lineal(Madrid)',
       'Concepción, Ciudad Lineal(Madrid)',
       'San Pascual, Ciudad Lineal(Madrid)', 'El Viso, Chamartín(Madrid)',
       'Nueva España, Chamartín(Madrid)', 'Castilla, Chamartín(Madrid)',
       'Bernabéu-Hispanoamérica, Chamartín(Madrid)',
       'Prosperidad, Chamartín(Madrid)',
       'Ciudad Jardín, Chamartín(Madrid)',
       'Lavapiés-Embajadores, Centro(Madrid)',
       'Opañel, Carabanchel(Madrid)', 'Comillas, Carabanchel(Madrid)',
       'Abrantes, Carabanchel(Madrid)', 'San Isidro, Carabanchel(Madrid)',
       'Vista Alegre, Carabanchel(Madrid)',
       'Pau de Carabanchel, Carabanchel(Madrid)',
       'Puerta Bonita, Carabanchel(Madrid)',
       #'Buena Vista, Carabanchel(Madrid)',
       'Huertas-Cortes, Centro(Madrid)',
       'Malasaña-Universidad, Centro(Madrid)',
       'Chueca-Justicia, Centro(Madrid)', 'Palacio, Centro(Madrid)',
       'Sol, Centro(Madrid)', 'Recoletos, Salamanca(Madrid)',
       'Goya, Salamanca(Madrid)', 'Lista, Salamanca(Madrid)',
       'Castellana, Salamanca(Madrid)', 'Guindalera, Salamanca(Madrid)',
       'Fuente del Berro, Salamanca(Madrid)',
       'Imperial, Arganzuela(Madrid)', 'Chopera, Arganzuela(Madrid)',
       'Acacias, Arganzuela(Madrid)',
       'Palos de Moguer, Arganzuela(Madrid)',
       'Legazpi, Arganzuela(Madrid)', 'Delicias, Arganzuela(Madrid)',
       'Alameda de Osuna, Barajas(Madrid)',
       #'Casco Histórico de Barajas, Barajas(Madrid)',
       #'Campo de las Naciones-Corralejos, Barajas(Madrid)',
       'Timón, Barajas(Madrid)']

In [23]:
geolocator = Nominatim(user_agent="http")

coordict = {}
for neigh in coord1:

        locate = geolocator.geocode(neigh, timeout=7)
        coordict[neigh]= [locate.latitude, locate.longitude]

In [24]:
# Buscamos manualmente las coordenadas de los barrios que geopy no localizaba y los añadimos al diccionario.

coordict.update({'Valdebernardo - Valderribas, Vicálvaro(Madrid)': [40.401583333333, -3.5990944444444],
                 'El Cañaveral - Los Berrocales, Vicálvaro(Madrid)': [40.404289, -3.5466159],
                 'Ensanche de Vallecas - La Gavia, Villa de Vallecas(Madrid)':[40.36923675, -3.61257634577546],
                 'Los Cármenes, Latina(Madrid)': [40.406944, -3.726389],
                 '12 de Octubre-Orcasur, Usera(Madrid)': [40.37415595, -3.69876507544298],
                 'Arroyo del Fresno, Fuencarral(Madrid)': [40.5009575, -3.7196907],
                 'Buena Vista, Carabanchel(Madrid)': [40.3728306, -3.7430863],
                 'Casco Histórico de Barajas, Barajas(Madrid)': [40.47984005, -3.5802507041958],
                 'Campo de las Naciones-Corralejos, Barajas(Madrid)': [40.4597489, -3.61273751083333]})
coordict

{'San Cristóbal, Villaverde(Madrid)': [40.343263050000004, -3.688352297394898],
 'Los Ángeles, Villaverde(Madrid)': [40.3551089, -3.7001047141537544],
 'San Andrés, Villaverde(Madrid)': [40.3455389, -3.7109697],
 'Los Rosales, Villaverde(Madrid)': [40.3558406, -3.688398889638639],
 'Butarque, Villaverde(Madrid)': [40.3399339, -3.6733878585167616],
 'Ambroz, Vicálvaro(Madrid)': [40.4058276, -3.6051718],
 'Casco Histórico de Vicálvaro, Vicálvaro(Madrid)': [40.4052863, -3.6072004],
 'Casco Histórico de Vallecas, Villa de Vallecas(Madrid)': [40.3773477,
  -3.6217118],
 'Santa Eugenia, Villa de Vallecas(Madrid)': [40.3829244, -3.6117674],
 'Orcasitas, Usera(Madrid)': [40.36727875, -3.712509095606202],
 'San Fermín, Usera(Madrid)': [40.3719909, -3.6903920331665687],
 'Moscardó, Usera(Madrid)': [40.3900156, -3.7054243359777326],
 'Pradolongo, Usera(Madrid)': [40.37863215, -3.705950189299193],
 'Almendrales, Usera(Madrid)': [40.3836929, -3.699095868824385],
 'Zofío, Usera(Madrid)': [40.3798077

In [25]:
# Obtengo la columna latitud.

latitude = []
for loc in sales_houses_clean["neighborhood_id"]:
    for key, value in coordict.items():
        if key == loc:
            latitude.append(value[0])
            
sales_houses_clean["latitude"] = pd.DataFrame(latitude)

In [26]:
# Obtengo la columna longitude.

longitude = []
for loc in sales_houses_clean["neighborhood_id"]:
    for key, value in coordict.items():
        if key == loc:
            longitude.append(value[1])
            
sales_houses_clean["longitude"] = pd.DataFrame(longitude)

In [27]:
# Vemos que los datos de nuestra tabla son los correctos.

sales_houses_clean.head()

,level_0,index,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,...,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area,latitude,longitude
0,0,0,"San Cristóbal, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,3,...,False,False,False,False,False,False,85000,1328,40.343263,-3.688352
1,1,1,"Los Ángeles, Villaverde(Madrid)",HouseType 1: Pisos,True,False,70.0,3,1.0,4,...,True,False,False,True,False,False,129900,1856,40.355109,-3.700105
2,2,2,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,94.0,2,2.0,1,...,True,False,True,False,False,False,144247,1535,40.345539,-3.710970
3,3,3,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,...,True,False,True,False,False,False,109900,1717,40.345539,-3.710970
4,4,4,"Los Rosales, Villaverde(Madrid)",HouseType 1: Pisos,False,False,108.0,2,2.0,4,...,True,True,True,False,False,True,260000,2407,40.355841,-3.688399


In [28]:
# Eliminamos las columnas "level_0" e "index".

sales_houses_clean = sales_houses_clean.drop(["level_0", "index"], axis=1)

In [29]:
# Exportamos el archivo que utilizaremos para nuestro análisis.

sales_houses_clean.to_csv("../outputs/clean_houses.csv")